In [1]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn
import sys

/home/alexander_mpa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# loading model data 

dem_train = pd.read_csv("data/political.train.0", delimiter="\t", header = None, names = ['sentence_text'])
rep_train = pd.read_csv("data/political.train.1", delimiter="\t", header = None, names = ['sentence_text'])
dem_test = pd.read_csv("data/political.test.0", delimiter="\t", header = None, names = ['sentence_text'])
rep_test = pd.read_csv("data/political.test.1", delimiter="\t", header = None, names = ['sentence_text'])


dem_rep_style_test = pd.DataFrame(data = open("data/political.test.0.tsf.dem-rep").readlines(), columns = ['sentence_text'])

rep_dem_style_test = pd.DataFrame(data = open("data/political.test.1.tsf.rep-dem").readlines(), columns = ['sentence_text'])

dem_train['party']='democratic'
dem_test['party']='democratic'
rep_train['party']='republican'
rep_test['party']='republican'

dem_rep_style_test['party']='republican'
rep_dem_style_test['party']='democratic'

#combine dem/rep data together for training/testing
#shuffle it for random order
all_train = dem_train.append(rep_train)
all_test = dem_test.append(rep_test)

all_style_test = dem_rep_style_test.append(rep_dem_style_test)

all_train = all_train.sample(frac=.25)
all_test = all_test.sample(frac=.1)

all_style_test = all_style_test.sample(frac=.1)

print(all_train.head())
print(all_test.head())
print(all_style_test.head())

                                            sentence_text       party
243865                           democratic thanks mark .  democratic
101886  democratic he missed the best small business i...  democratic
172884  democratic congrats debbie , i know you will d...  democratic
121633  democratic she ' s an embarrassment every time...  democratic
100280  republican i think you are cutting your nose o...  republican
                                           sentence_text       party
20226           i don ' t know about that , sen booker .  democratic
4636             we need your voice now more than ever !  democratic
27566              my guess is , it ' s an obama quote !  republican
27858  when are you going to stop voting for debt cei...  republican
3009   also , no federal aid to countries that do not...  republican
                                           sentence_text       party
8325       republican you are a good man , mr . paul .\n  republican
3571     democratic please g

In [3]:
train_x = all_train['sentence_text']
train_y = all_train['party']
test_x = all_test['sentence_text']
test_y = all_test['party']

style_test_x = all_style_test['sentence_text']
style_test_y = all_style_test['party']

In [4]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

train_x_tokens = []
test_x_tokens = []
style_test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))
    
for x in style_test_x:
    style_test_x_tokens.append(tokenizer.tokenize(x))

In [5]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []
style_test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))
    
for x in style_test_x_tokens:
    style_test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 20,000
Vocabulary dict:  {'<s>': 0, '</s>': 1, '<unk>': 2, '.': 3, 'the': 4, 'republican': 5, 'democratic': 6, ',': 7, 'to': 8, 'and': 9, 'you': 10, '!': 11, 'of': 12, 'for': 13, 'a': 14, "'": 15, 'is': 16, 'i': 17, 'in': 18, 'that': 19, 'are': 20, 'it': 21, 'your': 22, 'we': 23, 'this': 24, 'not': 25, 'have': 26, 's': 27, '?': 28, 'on': 29, 'with': 30, 'be': 31, 'our': 32, 't': 33, 'all': 34, '"': 35, 'he': 36, 'will': 37, 'do': 38, 'as': 39, 'senator': 40, 'people': 41, 'obama': 42, 'what': 43, 'no': 44, 'so': 45, 'but': 46, 'if': 47, 'from': 48, 'has': 49, 'they': 50, '-': 51, 'can': 52, 'vote': 53, 'thank': 54, 'us': 55, 'who': 56, 'about': 57, 'like': 58, 'get': 59, '/': 60, 'by': 61, 'would': 62, 'up': 63, 'or': 64, 'his': 65, 'my': 66, 'out': 67, 'was': 68, 'trump': 69, 'president': 70, 'just': 71, 'please': 72, 'need': 73, 'more': 74, 'don': 75, 'warren': 76, 'at': 77, 'should': 78, 'an': 79, 'when': 80, 'why': 81, 'one': 82, 'their': 83, 'country': 84, 'now': 

In [6]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []
style_test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))
    
for x in style_test_x_ids:
    # remove democrat/republican tag from style test data
    del x[0]
    style_test_x_fdict.append(collections.Counter(x))
    
train_x_vector = []
test_x_vector = []
style_test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in style_test_x_fdict:
    style_test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)
style_test_x_sparse_bow = utils.id_lists_to_sparse_bow(style_test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)
y_pred_style = nb.predict(style_test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

acc_style = accuracy_score(style_test_y, y_pred_style)
print("Accuracy on style test set: {:.02%}".format(acc_style))
classrep_style = classification_report(style_test_y, y_pred_style)
print(classrep_style)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 98.29%
             precision    recall  f1-score   support

 democratic       0.98      0.98      0.98      2822
 republican       0.98      0.98      0.98      2778

avg / total       0.98      0.98      0.98      5600

Accuracy on style test set: 83.46%
             precision    recall  f1-score   support

 democratic       0.83      0.84      0.84      2811
 republican       0.84      0.83      0.83      2789

avg / total       0.83      0.83      0.83      5600



In [7]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids
style_test_x_neural = style_test_x_ids

for x in train_x_neural:
    # remove democrat/republican tag from train data
    del x[0]
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in style_test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [['democratic'],['republican']]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())
style_test_labels = np.array(style_test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]
style_test_lables_format = [style_test_labels[i:i+1] for i in range(0, len(style_test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)
style_test_y_neural = lb.transform(test_lables_format)

In [32]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 256, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 50, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2c31296fd0>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f2c31296ba8>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f2c31296da0>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f2c31296a90>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f2c312969b0>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f2c31296cf8>
F=<absl.flags._flag.Flag object at 0x7f2c312a1080>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f2c31296940>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f2c31296b00>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2c31296f98>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f2c31296e48>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f2c31296c50>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f2c31296a20>

x_train (134480, 195)
y_train (134480, 2)
x_dev (5600, 195)
x_dev (5600, 2)
Vocabulary Size: 20000
Train/Dev split: 134480/5600
INFO:tensorflow:Summar

2018-08-06T18:27:03.375476: step 69, loss 1.35004, acc 0.6875
2018-08-06T18:27:03.506789: step 70, loss 1.40006, acc 0.59375
2018-08-06T18:27:03.642759: step 71, loss 2.08092, acc 0.59375
2018-08-06T18:27:03.778442: step 72, loss 1.68986, acc 0.625
2018-08-06T18:27:03.909864: step 73, loss 1.57893, acc 0.625
2018-08-06T18:27:04.047930: step 74, loss 1.21091, acc 0.671875
2018-08-06T18:27:04.180771: step 75, loss 1.80381, acc 0.53125
2018-08-06T18:27:04.315370: step 76, loss 1.62508, acc 0.65625
2018-08-06T18:27:04.447693: step 77, loss 0.868629, acc 0.734375
2018-08-06T18:27:04.580900: step 78, loss 1.5572, acc 0.578125
2018-08-06T18:27:04.714064: step 79, loss 2.05523, acc 0.5625
2018-08-06T18:27:04.840395: step 80, loss 1.19972, acc 0.671875
2018-08-06T18:27:04.973618: step 81, loss 1.89997, acc 0.578125
2018-08-06T18:27:05.101216: step 82, loss 1.4142, acc 0.640625
2018-08-06T18:27:05.234182: step 83, loss 1.3597, acc 0.640625
2018-08-06T18:27:05.362411: step 84, loss 1.45363, acc 0

2018-08-06T18:27:21.350937: step 195, loss 0.99697, acc 0.640625
2018-08-06T18:27:21.476721: step 196, loss 0.909615, acc 0.71875
2018-08-06T18:27:21.611327: step 197, loss 0.517386, acc 0.8125
2018-08-06T18:27:21.742095: step 198, loss 0.920755, acc 0.6875
2018-08-06T18:27:21.874452: step 199, loss 0.634355, acc 0.75
2018-08-06T18:27:22.006641: step 200, loss 0.918221, acc 0.703125

Evaluation:
2018-08-06T18:27:23.040462: step 200, loss 0.441396, acc 0.8075

Saved model checkpoint to /home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/model-200

2018-08-06T18:27:23.323898: step 201, loss 0.633956, acc 0.765625
2018-08-06T18:27:23.452174: step 202, loss 0.960287, acc 0.71875
2018-08-06T18:27:23.587596: step 203, loss 1.056, acc 0.734375
2018-08-06T18:27:23.718385: step 204, loss 1.00885, acc 0.640625
2018-08-06T18:27:23.848042: step 205, loss 0.815859, acc 0.75
2018-08-06T18:27:23.982656: step 206, loss 0.752641, acc 0.8125
2018-08-06T18:27:24.115914: st

2018-08-06T18:27:39.806675: step 317, loss 0.667873, acc 0.75
2018-08-06T18:27:39.940487: step 318, loss 0.742038, acc 0.78125
2018-08-06T18:27:40.073737: step 319, loss 0.375782, acc 0.859375
2018-08-06T18:27:40.204560: step 320, loss 0.526022, acc 0.828125
2018-08-06T18:27:40.340046: step 321, loss 0.443277, acc 0.84375
2018-08-06T18:27:40.472599: step 322, loss 0.664278, acc 0.8125
2018-08-06T18:27:40.600591: step 323, loss 0.31599, acc 0.84375
2018-08-06T18:27:40.736314: step 324, loss 0.555848, acc 0.796875
2018-08-06T18:27:40.866589: step 325, loss 0.403014, acc 0.859375
2018-08-06T18:27:40.993096: step 326, loss 0.856169, acc 0.703125
2018-08-06T18:27:41.119571: step 327, loss 0.572325, acc 0.765625
2018-08-06T18:27:41.248910: step 328, loss 0.714522, acc 0.78125
2018-08-06T18:27:41.375358: step 329, loss 0.449394, acc 0.78125
2018-08-06T18:27:41.504304: step 330, loss 0.956268, acc 0.71875
2018-08-06T18:27:41.631986: step 331, loss 0.578753, acc 0.8125
2018-08-06T18:27:41.75770

2018-08-06T18:27:56.933974: step 441, loss 0.456402, acc 0.875
2018-08-06T18:27:57.063167: step 442, loss 0.252603, acc 0.875
2018-08-06T18:27:57.191861: step 443, loss 0.468752, acc 0.796875
2018-08-06T18:27:57.324972: step 444, loss 0.393984, acc 0.84375
2018-08-06T18:27:57.455005: step 445, loss 0.494323, acc 0.78125
2018-08-06T18:27:57.583822: step 446, loss 0.552752, acc 0.796875
2018-08-06T18:27:57.709434: step 447, loss 0.306534, acc 0.859375
2018-08-06T18:27:57.837568: step 448, loss 0.281524, acc 0.90625
2018-08-06T18:27:57.965066: step 449, loss 0.214681, acc 0.90625
2018-08-06T18:27:58.092308: step 450, loss 0.224404, acc 0.90625
2018-08-06T18:27:58.215920: step 451, loss 0.413708, acc 0.84375
2018-08-06T18:27:58.345952: step 452, loss 0.418447, acc 0.8125
2018-08-06T18:27:58.473199: step 453, loss 0.17656, acc 0.96875
2018-08-06T18:27:58.602929: step 454, loss 0.462099, acc 0.875
2018-08-06T18:27:58.731611: step 455, loss 0.464749, acc 0.8125
2018-08-06T18:27:58.858150: ste

...

2018-08-06T18:41:44.783629: step 6225, loss 0.149194, acc 0.9375
2018-08-06T18:41:44.923135: step 6226, loss 0.00144986, acc 1
2018-08-06T18:41:45.054178: step 6227, loss 0.000262151, acc 1
2018-08-06T18:41:45.186160: step 6228, loss 0.0297218, acc 0.984375
2018-08-06T18:41:45.322125: step 6229, loss 0.000338418, acc 1
2018-08-06T18:41:45.458302: step 6230, loss 0.0739872, acc 0.96875
2018-08-06T18:41:45.594533: step 6231, loss 0.000142668, acc 1
2018-08-06T18:41:45.730339: step 6232, loss 0.0483867, acc 0.984375
2018-08-06T18:41:45.864953: step 6233, loss 0.000247011, acc 1
2018-08-06T18:41:45.992791: step 6234, loss 0.0124479, acc 1
2018-08-06T18:41:46.119602: step 6235, loss 0.000263511, acc 1
2018-08-06T18:41:46.254519: step 6236, loss 0.00172093, acc 1
2018-08-06T18:41:46.384226: step 6237, loss 0.125947, acc 0.984375
2018-08-06T18:41:46.523163: step 6238, loss 0.125369, acc 0.984375
2018-08-06T18:41:46.656141: step 6239, loss 3.98016e-06, acc 1
2018-08-06T18:41:46.791995: step 62

2018-08-06T18:42:02.594240: step 6351, loss 0.114901, acc 0.96875
2018-08-06T18:42:02.719456: step 6352, loss 0.0817906, acc 0.984375
2018-08-06T18:42:02.847380: step 6353, loss 2.84304e-05, acc 1
2018-08-06T18:42:02.969914: step 6354, loss 8.64995e-05, acc 1
2018-08-06T18:42:03.093777: step 6355, loss 0.00250036, acc 1
2018-08-06T18:42:03.220967: step 6356, loss 0.000187561, acc 1
2018-08-06T18:42:03.347488: step 6357, loss 0.00913406, acc 1
2018-08-06T18:42:03.477557: step 6358, loss 6.99666e-05, acc 1
2018-08-06T18:42:03.603632: step 6359, loss 4.63178e-05, acc 1
2018-08-06T18:42:03.725073: step 6360, loss 0.000482249, acc 1
2018-08-06T18:42:03.850128: step 6361, loss 0.284259, acc 0.96875
2018-08-06T18:42:03.974855: step 6362, loss 0.00191682, acc 1
2018-08-06T18:42:04.103929: step 6363, loss 0.0309567, acc 0.984375
2018-08-06T18:42:04.226303: step 6364, loss 5.76325e-05, acc 1
2018-08-06T18:42:04.354854: step 6365, loss 0.00101108, acc 1
2018-08-06T18:42:04.479760: step 6366, loss

2018-08-06T18:42:20.269182: step 6477, loss 0.000281037, acc 1
2018-08-06T18:42:20.398303: step 6478, loss 0.0309161, acc 0.984375
2018-08-06T18:42:20.526384: step 6479, loss 0.0240728, acc 0.984375
2018-08-06T18:42:20.651931: step 6480, loss 1.71006e-05, acc 1
2018-08-06T18:42:20.779860: step 6481, loss 0.00812616, acc 1
2018-08-06T18:42:20.901921: step 6482, loss 0.0156955, acc 0.984375
2018-08-06T18:42:21.032042: step 6483, loss 6.2464e-06, acc 1
2018-08-06T18:42:21.160803: step 6484, loss 2.08921e-05, acc 1
2018-08-06T18:42:21.294302: step 6485, loss 0.00134128, acc 1
2018-08-06T18:42:21.424640: step 6486, loss 2.28957e-05, acc 1
2018-08-06T18:42:21.557600: step 6487, loss 0.0364793, acc 0.984375
2018-08-06T18:42:21.690762: step 6488, loss 0.106993, acc 0.984375
2018-08-06T18:42:21.825742: step 6489, loss 0.0144994, acc 0.984375
2018-08-06T18:42:21.960595: step 6490, loss 3.10905e-05, acc 1
2018-08-06T18:42:22.093243: step 6491, loss 0.0100703, acc 1
2018-08-06T18:42:22.219120: ste

2018-08-06T18:42:38.956135: step 6601, loss 1.18531e-05, acc 1
2018-08-06T18:42:39.081986: step 6602, loss 0.00184997, acc 1
2018-08-06T18:42:39.210164: step 6603, loss 0.0318361, acc 0.984375
2018-08-06T18:42:39.337077: step 6604, loss 0.000197147, acc 1
2018-08-06T18:42:39.463654: step 6605, loss 0.0769002, acc 0.984375
2018-08-06T18:42:39.589637: step 6606, loss 4.77341e-06, acc 1
2018-08-06T18:42:39.711172: step 6607, loss 2.2853e-05, acc 1
2018-08-06T18:42:39.838637: step 6608, loss 1.35076e-05, acc 1
2018-08-06T18:42:39.964117: step 6609, loss 0.000530975, acc 1
2018-08-06T18:42:40.098542: step 6610, loss 1.89237e-06, acc 1
2018-08-06T18:42:40.221818: step 6611, loss 2.85898e-06, acc 1
2018-08-06T18:42:40.350784: step 6612, loss 2.38593e-06, acc 1
2018-08-06T18:42:40.483426: step 6613, loss 5.6932e-05, acc 1
2018-08-06T18:42:40.613141: step 6614, loss 2.03027e-07, acc 1
2018-08-06T18:42:40.746829: step 6615, loss 0.000104973, acc 1
2018-08-06T18:42:40.879953: step 6616, loss 4.28

KeyboardInterrupt: 

In [53]:
# running CNN model against style transferred test data

# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

# Eval Parameters
# Update these checkpoint locations of the latest model from the log output of the cell above
tf.flags.DEFINE_string("checkpoint_dir", "/home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/", "Checkpoint directory from training run")
checkpoint_file = "/home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/model-6700"

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

x_test = np.array(style_test_x_neural)
y_test = np.array(style_test_y_neural)


print("\nEvaluating...\n")

# Evaluation
# ==================================================
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy

lb = preprocessing.LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
lb.fit(np.asarray(style_test_y.tolist()).flatten())  
style_test_y_neural_acc = lb.transform(np.asarray(style_test_y.tolist()))

acc_style_neural = accuracy_score(style_test_y_neural_acc, all_predictions)
print("Accuracy on style test set: {:.02%}".format(acc_style_neural))
classrep_style_neural = classification_report(style_test_y_neural_acc, all_predictions)
print(classrep_style)


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2c461699e8>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f2c30233ac8>
CHECKPOINT_DIR=<absl.flags._flag.Flag object at 0x7f2c461389b0>
F=<absl.flags._flag.Flag object at 0x7f2c4639e940>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f2c46169a20>


Evaluating...

INFO:tensorflow:Restoring parameters from /home/alexander_mpa/capstone/Parlancr/models/classifier/runs/1533580012/checkpoints/model-6700
Accuracy on style test set: 83.30%
             precision    recall  f1-score   support

 democratic       0.83      0.84      0.84      2811
 republican       0.84      0.83      0.83      2789

avg / total       0.83      0.83      0.83      5600

